# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [21]:
import tensorflow as tf
import pandas as pd
houses = pd.read_csv('cal_housing_clean.csv')

In [3]:
from sklearn.model_selection import train_test_split
houses.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [6]:
X = houses.drop('medianHouseValue',axis=1)
y = houses['medianHouseValue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [8]:
X_train.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
count,14448.000000,14448.000000,14448.000000,14448.000000,14448.000000,14448.000000
mean,28.569421,2626.209718,534.725568,1416.847661,496.624031,3.885479
std,12.603111,2145.659354,412.919305,1126.924943,373.922937,1.896295
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900
25%,18.000000,1443.750000,295.000000,786.750000,279.000000,2.574825
50%,29.000000,2122.000000,433.000000,1162.000000,408.000000,3.550500
75%,37.000000,3145.250000,644.000000,1718.000000,602.000000,4.775000
max,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100


In [9]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14448 entries, 3173 to 1895
Data columns (total 6 columns):
housingMedianAge    14448 non-null float64
totalRooms          14448 non-null float64
totalBedrooms       14448 non-null float64
population          14448 non-null float64
households          14448 non-null float64
medianIncome        14448 non-null float64
dtypes: float64(6)
memory usage: 790.1 KB


In [10]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6192 entries, 147 to 10960
Data columns (total 6 columns):
housingMedianAge    6192 non-null float64
totalRooms          6192 non-null float64
totalBedrooms       6192 non-null float64
population          6192 non-null float64
households          6192 non-null float64
medianIncome        6192 non-null float64
dtypes: float64(6)
memory usage: 338.6 KB


In [12]:
y_test.head()

147      227700.0
10782    325400.0
2969     245800.0
15412    205000.0
19986     63100.0
Name: medianHouseValue, dtype: float64

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [13]:
from sklearn.preprocessing import MinMaxScaler

In [15]:
scaler = MinMaxScaler()
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [16]:
scaled_train = scaler.transform(X_train)
scaled_test = scaler.transform(X_test)

In [19]:
training = pd.DataFrame(scaled_train,columns=X_train.columns)
testing = pd.DataFrame(scaled_test,columns=X_train.columns)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [20]:
training.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome'],
      dtype='object')

In [22]:
house_age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
population = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [23]:
feat_cols = [house_age,rooms,bedrooms,population,households,income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [24]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=20,num_epochs=2000,shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [77]:
dnn = tf.estimator.DNNRegressor(hidden_units=[10,20,20,20,10],feature_columns=feat_cols,optimizer='Adam')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mf/dz_yzn890t978h4w_mt6m4d40000gn/T/tmpw361a0h2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a3d77d8d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [ ]:
dnn.train(input_fn=input_func,steps=100000)

In [ ]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=20,num_epochs=1,shuffle=False)

In [70]:
results = dnn.evaluate(input_fn=eval_input_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-24T19:36:36Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/mf/dz_yzn890t978h4w_mt6m4d40000gn/T/tmpykcesa_q/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-24-19:36:38
INFO:tensorflow:Saving dict for global step 1000: average_loss = 11116518000.0, global_step = 1000, label/mean = 206050.5, loss = 222043490000.0, prediction/mean = 207882.48
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /var/folders/mf/dz_yzn890t978h4w_mt6m4d40000gn/T/tmpykcesa_q/model.ckpt-1000


In [71]:
results

{'average_loss': 11116518000.0,
 'label/mean': 206050.5,
 'loss': 222043490000.0,
 'prediction/mean': 207882.48,
 'global_step': 1000}

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [72]:
pred_func = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=10,num_epochs=1,shuffle=False)

In [73]:
preds = dnn.predict(pred_func)

In [74]:
predictions = list(preds)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/mf/dz_yzn890t978h4w_mt6m4d40000gn/T/tmpykcesa_q/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [75]:
final_preds = []
for pred in predictions:
    print(pred)
    final_preds.append(pred['predictions'])

{'predictions': array([256909.], dtype=float32)}
{'predictions': array([191551.6], dtype=float32)}
{'predictions': array([295626.75], dtype=float32)}
{'predictions': array([240081.1], dtype=float32)}
{'predictions': array([161266.14], dtype=float32)}
{'predictions': array([337890.56], dtype=float32)}
{'predictions': array([138933.4], dtype=float32)}
{'predictions': array([237877.64], dtype=float32)}
{'predictions': array([179935.52], dtype=float32)}
{'predictions': array([144507.56], dtype=float32)}
{'predictions': array([199246.11], dtype=float32)}
{'predictions': array([277329.8], dtype=float32)}
{'predictions': array([137852.3], dtype=float32)}
{'predictions': array([188698.95], dtype=float32)}
{'predictions': array([321217.34], dtype=float32)}
{'predictions': array([271013.94], dtype=float32)}
{'predictions': array([199422.75], dtype=float32)}
{'predictions': array([198968.55], dtype=float32)}
{'predictions': array([177156.8], dtype=float32)}
{'predictions': array([189778.94], dtyp

{'predictions': array([148344.94], dtype=float32)}
{'predictions': array([140156.58], dtype=float32)}
{'predictions': array([106710.45], dtype=float32)}
{'predictions': array([245089.94], dtype=float32)}
{'predictions': array([214438.95], dtype=float32)}
{'predictions': array([216842.73], dtype=float32)}
{'predictions': array([165801.12], dtype=float32)}
{'predictions': array([154073.12], dtype=float32)}
{'predictions': array([173822.8], dtype=float32)}
{'predictions': array([206767.7], dtype=float32)}
{'predictions': array([200249.27], dtype=float32)}
{'predictions': array([156136.55], dtype=float32)}
{'predictions': array([379421.53], dtype=float32)}
{'predictions': array([216075.77], dtype=float32)}
{'predictions': array([160387.97], dtype=float32)}
{'predictions': array([169591.64], dtype=float32)}
{'predictions': array([132897.97], dtype=float32)}
{'predictions': array([253374.11], dtype=float32)}
{'predictions': array([230416.42], dtype=float32)}
{'predictions': array([136280.5],

{'predictions': array([184814.27], dtype=float32)}
{'predictions': array([205819.3], dtype=float32)}
{'predictions': array([202964.75], dtype=float32)}
{'predictions': array([200675.34], dtype=float32)}
{'predictions': array([146578.45], dtype=float32)}
{'predictions': array([220937.72], dtype=float32)}
{'predictions': array([215535.2], dtype=float32)}
{'predictions': array([247258.69], dtype=float32)}
{'predictions': array([263167.38], dtype=float32)}
{'predictions': array([178709.77], dtype=float32)}
{'predictions': array([96659.38], dtype=float32)}
{'predictions': array([162507.61], dtype=float32)}
{'predictions': array([222739.19], dtype=float32)}
{'predictions': array([161652.06], dtype=float32)}
{'predictions': array([124265.95], dtype=float32)}
{'predictions': array([281110.28], dtype=float32)}
{'predictions': array([710018.5], dtype=float32)}
{'predictions': array([135649.48], dtype=float32)}
{'predictions': array([227112.16], dtype=float32)}
{'predictions': array([274304.03], 

{'predictions': array([294958.2], dtype=float32)}
{'predictions': array([182781.72], dtype=float32)}
{'predictions': array([211079.19], dtype=float32)}
{'predictions': array([240549.78], dtype=float32)}
{'predictions': array([164260.92], dtype=float32)}
{'predictions': array([206510.81], dtype=float32)}
{'predictions': array([171856.61], dtype=float32)}
{'predictions': array([273007.2], dtype=float32)}
{'predictions': array([193143.89], dtype=float32)}
{'predictions': array([169429.05], dtype=float32)}
{'predictions': array([213821.38], dtype=float32)}
{'predictions': array([230736.56], dtype=float32)}
{'predictions': array([180709.08], dtype=float32)}
{'predictions': array([234387.56], dtype=float32)}
{'predictions': array([253304.62], dtype=float32)}
{'predictions': array([269528.3], dtype=float32)}
{'predictions': array([121205.164], dtype=float32)}
{'predictions': array([181909.81], dtype=float32)}
{'predictions': array([197056.39], dtype=float32)}
{'predictions': array([183534.14]

{'predictions': array([186114.28], dtype=float32)}
{'predictions': array([256674.06], dtype=float32)}
{'predictions': array([144509.31], dtype=float32)}
{'predictions': array([241757.58], dtype=float32)}
{'predictions': array([253104.73], dtype=float32)}
{'predictions': array([369183.1], dtype=float32)}
{'predictions': array([240047.81], dtype=float32)}
{'predictions': array([229361.88], dtype=float32)}
{'predictions': array([222342.06], dtype=float32)}
{'predictions': array([239989.17], dtype=float32)}
{'predictions': array([238005.1], dtype=float32)}
{'predictions': array([173276.94], dtype=float32)}
{'predictions': array([154521.97], dtype=float32)}
{'predictions': array([215590.33], dtype=float32)}
{'predictions': array([197503.52], dtype=float32)}
{'predictions': array([241857.92], dtype=float32)}
{'predictions': array([182185.38], dtype=float32)}
{'predictions': array([243038.22], dtype=float32)}
{'predictions': array([219833.03], dtype=float32)}
{'predictions': array([104776.195

...

{'predictions': array([245420.94], dtype=float32)}
{'predictions': array([126517.305], dtype=float32)}
{'predictions': array([254564.62], dtype=float32)}
{'predictions': array([190598.56], dtype=float32)}
{'predictions': array([182772.94], dtype=float32)}
{'predictions': array([201822.69], dtype=float32)}
{'predictions': array([226200.7], dtype=float32)}
{'predictions': array([139250.6], dtype=float32)}
{'predictions': array([225918.25], dtype=float32)}
{'predictions': array([166776.38], dtype=float32)}
{'predictions': array([151446.4], dtype=float32)}
{'predictions': array([405734.6], dtype=float32)}
{'predictions': array([348342.3], dtype=float32)}
{'predictions': array([181145.25], dtype=float32)}
{'predictions': array([156727.39], dtype=float32)}
{'predictions': array([180912.92], dtype=float32)}
{'predictions': array([105544.51], dtype=float32)}
{'predictions': array([234530.08], dtype=float32)}
{'predictions': array([163758.58], dtype=float32)}
{'predictions': array([137881.92], 

{'predictions': array([205733.22], dtype=float32)}
{'predictions': array([306550.9], dtype=float32)}
{'predictions': array([179346.17], dtype=float32)}
{'predictions': array([183323.89], dtype=float32)}
{'predictions': array([194666.22], dtype=float32)}
{'predictions': array([131810.72], dtype=float32)}
{'predictions': array([265061.03], dtype=float32)}
{'predictions': array([367036.38], dtype=float32)}
{'predictions': array([173182.78], dtype=float32)}
{'predictions': array([252207.52], dtype=float32)}
{'predictions': array([145297.03], dtype=float32)}
{'predictions': array([199437.89], dtype=float32)}
{'predictions': array([139138.19], dtype=float32)}
{'predictions': array([200958.], dtype=float32)}
{'predictions': array([113055.56], dtype=float32)}
{'predictions': array([186752.61], dtype=float32)}
{'predictions': array([181076.69], dtype=float32)}
{'predictions': array([304432.9], dtype=float32)}
{'predictions': array([271617.47], dtype=float32)}
{'predictions': array([190254.98], 

{'predictions': array([246099.8], dtype=float32)}
{'predictions': array([289046.25], dtype=float32)}
{'predictions': array([167171.75], dtype=float32)}
{'predictions': array([223037.7], dtype=float32)}
{'predictions': array([97672.914], dtype=float32)}
{'predictions': array([175315.88], dtype=float32)}
{'predictions': array([331442.5], dtype=float32)}
{'predictions': array([162867.7], dtype=float32)}
{'predictions': array([217107.22], dtype=float32)}
{'predictions': array([168236.08], dtype=float32)}
{'predictions': array([214539.34], dtype=float32)}
{'predictions': array([166138.64], dtype=float32)}
{'predictions': array([210299.34], dtype=float32)}
{'predictions': array([236873.14], dtype=float32)}
{'predictions': array([151918.14], dtype=float32)}
{'predictions': array([167479.31], dtype=float32)}
{'predictions': array([206323.56], dtype=float32)}
{'predictions': array([143416.52], dtype=float32)}
{'predictions': array([199039.5], dtype=float32)}
{'predictions': array([129709.07], d

{'predictions': array([206799.16], dtype=float32)}
{'predictions': array([202093.34], dtype=float32)}
{'predictions': array([374285.22], dtype=float32)}
{'predictions': array([171157.73], dtype=float32)}
{'predictions': array([204232.83], dtype=float32)}
{'predictions': array([103828.31], dtype=float32)}
{'predictions': array([143251.1], dtype=float32)}
{'predictions': array([149073.78], dtype=float32)}
{'predictions': array([220577.02], dtype=float32)}
{'predictions': array([154804.88], dtype=float32)}
{'predictions': array([190252.36], dtype=float32)}
{'predictions': array([338413.16], dtype=float32)}
{'predictions': array([196288.89], dtype=float32)}
{'predictions': array([186637.16], dtype=float32)}
{'predictions': array([172047.52], dtype=float32)}
{'predictions': array([292105.38], dtype=float32)}
{'predictions': array([148083.89], dtype=float32)}
{'predictions': array([177608.12], dtype=float32)}
{'predictions': array([266012.6], dtype=float32)}
{'predictions': array([202697.27]

{'predictions': array([181600.86], dtype=float32)}
{'predictions': array([234108.47], dtype=float32)}
{'predictions': array([179810.3], dtype=float32)}
{'predictions': array([231861.73], dtype=float32)}
{'predictions': array([244457.02], dtype=float32)}
{'predictions': array([242365.67], dtype=float32)}
{'predictions': array([189667.73], dtype=float32)}
{'predictions': array([218244.14], dtype=float32)}
{'predictions': array([288390.06], dtype=float32)}
{'predictions': array([232600.2], dtype=float32)}
{'predictions': array([201419.84], dtype=float32)}
{'predictions': array([130539.75], dtype=float32)}
{'predictions': array([292576.3], dtype=float32)}
{'predictions': array([280301.56], dtype=float32)}
{'predictions': array([189928.19], dtype=float32)}
{'predictions': array([173636.44], dtype=float32)}
{'predictions': array([153235.95], dtype=float32)}
{'predictions': array([218225.3], dtype=float32)}
{'predictions': array([132681.12], dtype=float32)}
{'predictions': array([96059.64], d

In [76]:
from sklearn.metrics import mean_squared_error
import numpy as np
print(np.sqrt(mean_squared_error(y_test,final_preds)))

105434.89103106732


# Great Job!